In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

%matplotlib inline

In [2]:
from person import Person
from externalities import World, Categorical, Offer

In [3]:
world = World()

In [4]:
Offer.print_help()

timestamp: int between 0 and 999999999
valid_from: int between 0 and 999999999
valid_to: int between 0 and 999999999
difficulty: positive numeric
reward: positive numeric
channel: Categorical(['web' 'email' 'mobile' 'social'])
type: Categorical(['bogo' 'discount' 'informational'])


In [5]:
offer_channel = Categorical(('web', 'email', 'mobile', 'social'), (1, 1, 1, 1))
offer_type = Categorical(('bogo', 'discount', 'informational'), (0, 1, 0))

discount = Offer(0, valid_from=10, valid_until=20, difficulty=10, reward=2, channel=offer_channel, type=offer_type)

In [6]:
Person.print_help()

id: uuid.uuid4() (set automatically)
dob: 'YYYYMMDD'
gender: ['M', 'F', 'O']
became_member_on: 'YYYYMMDD'
income: positive numeric
taste: Categorical(['sweet','sour','salty','bitter','umami'])
marketing_segment: Categorical(['front page','local','entertainment','sports','opinion','comics'])

last_transaction: Transaction
last_unviewed_offer: Offer
last_viewed_offer: Offer
history: list of Offers and Transactions sorted by Event timestamps

view_offer_sensitivity: Categorical(['background','offer_age','web','email','mobile','social'])
make_purchase_sensitivity: Categorical(['background','time_since_last_transaction','last_viewed_offer_strength','viewed_active_offer'])
purchase_amount_sensitivity: Categorical(['background','income_adjusted_purchase_sensitivity','front page','local','entertainment','sports','opinion','comics','sweet','sour','salty','bitter','umami'])


In [7]:
person_taste = Categorical(['sweet','sour','salty','bitter','umami'],
                           [1,1,0,0,0])

person_marketing_segment = Categorical(['front page','local','entertainment','sports','opinion','comics'],
                                       [1, 1, 0, 0, 0, 1])

person_view_offer_sensitivity = Categorical(['background', 'offer_age','web','email','mobile', 'social'],
                                            [0, -1, 1, 1, 1, 1])

person_make_purchase_sensitivity = Categorical(['background', 'time_since_last_transaction','time_since_last_viewed_offer','viewed_active_offer','web','email','mobile'])
person_purchase_amount_sensitivity = Categorical(['background','income_adjusted_purchase_sensitivity','front page','local','entertainment','sports','opinion','comics','sweet','sour','salty','bitter','umami'])

person = Person(became_member_on='20170716', view_offer_sensitivity=person_view_offer_sensitivity)
person.last_unviewed_offer = discount

In [8]:
discount.__dict__

{'difficulty': 10,
 'reward': 2,
 'timestamp': 0,
 'valid_from': 10,
 'valid_until': 20}

In [9]:
person.__dict__

{'became_member_on': '20170716',
 'dob': '19010101',
 'gender': None,
 'history': [],
 'id': UUID('288c688f-de07-4d84-9177-70bad410c424'),
 'income': None,
 'last_transaction': None,
 'last_unviewed_offer': <externalities.Offer at 0x99b1978>,
 'last_viewed_offer': None,
 'make_purchase_sensitivity': <externalities.Categorical at 0x9a56860>,
 'marketing_segment': <externalities.Categorical at 0x9a567b8>,
 'purchase_amount_sensitivity': <externalities.Categorical at 0x9a56908>,
 'taste': <externalities.Categorical at 0x9a56748>,
 'view_offer_sensitivity': <externalities.Categorical at 0x9a566a0>}

In [10]:
def g(x):
    return 1.0/(1.0 + np.exp(-x))

def g_inv(y):
    return np.log(y/(1.0-y))

In [11]:
g_inv(0.01)/float(1*24*30)

-0.0063821109029647081

In [12]:
for i in range(-5,5):
    print i, g(i+1) - g(i)

-5 0.0112933590378
-4 0.0294396632155
-3 0.0717770488446
-2 0.149738499348
-1 0.23105857863
0 0.23105857863
1 0.149738499348
2 0.0717770488446
3 0.0294396632155
4 0.0112933590378


In [53]:
offer_channel = Categorical(('web', 'email', 'mobile', 'social'), 
                            (1, 1, 1, 1))
offer_type = Categorical(('bogo', 'discount', 'informational'), 
                         (0, 1, 0))

discount = Offer(0, valid_from=10, valid_until=20, difficulty=10, reward=2, channel=offer_channel, type=offer_type)

person_view_offer_sensitivity = Categorical(['background', 
                                             'offer_age',
                                             'web','email','mobile', 'social'],
                                            [g_inv(0.20) - 4, 
                                             -abs(g_inv(0.01)/float(1*24*30)), 
                                             0, 1, 1, 2])

person_make_purchase_sensitivity = Categorical(['background', 
                                                'time_since_last_transaction',
                                                'last_viewed_offer_strength',
                                                'viewed_active_offer'],
                                               [g_inv(1.0/16.0),
                                                abs(g_inv(0.10)/float(1*24*30)),
                                                1,
                                                1])

person_purchase_amount_sensitivity = Categorical(['background',
                                                  'income_adjusted_purchase_sensitivity',
                                                  'front page',
                                                  'local',
                                                  'entertainment',
                                                  'sports',
                                                  'opinion',
                                                  'comics',
                                                  'sweet',
                                                  'sour',
                                                  'salty',
                                                  'bitter',
                                                  'umami'],
                                                [1,
                                                 0,
                                                 0,
                                                 0,
                                                 0,
                                                 0,
                                                 0,
                                                 0,
                                                 0,
                                                 0,
                                                 0,
                                                 0,
                                                 0])

person = Person(became_member_on='20170716', 
                view_offer_sensitivity=person_view_offer_sensitivity,
                make_purchase_sensitivity=person_make_purchase_sensitivity,
                purchase_amount_sensitivity=person_purchase_amount_sensitivity)
person.last_unviewed_offer = discount

world.world_time = 0

if person.view_offer(world) is not None:
    print 'Viewed'
else:
    print 'Not viewed'

history = list()
for i in range(100):
    world.world_time = i
    transaction = person.make_purchase(world)
    if transaction:
        #print i, 'Transaction'
        print i, transaction.__dict__
        history.append(1)
    else:
        print i, 'No Transaction'
        # print transaction
        history.append(0)
        
np.mean(history)

Viewed
----
[-2.7080502  0.0030517  1.         1.       ]
[ 1.  0.  1.  0.]
[-2.7080502  0.         1.         0.       ]
-1.7080502011
0.153416784696
0 No Transaction
----
[-2.7080502  0.0030517  1.         1.       ]
[ 1.          0.          0.98913043  0.        ]
[-2.7080502   0.          0.98913043  0.        ]
-1.71891976632
0.152010356906
1 No Transaction
----
[-2.7080502  0.0030517  1.         1.       ]
[ 1.          0.          0.97826087  0.        ]
[-2.7080502   0.          0.97826087  0.        ]
-1.72978933154
0.150614528535
2 {'timestamp': 2, 'amount': None}
----
[-2.7080502  0.0030517  1.         1.       ]
[ 1.         1.         0.9673913  0.       ]
[-2.7080502  0.0030517  0.9673913  0.       ]
-1.73760719595
0.149617120213
3 No Transaction
----
[-2.7080502  0.0030517  1.         1.       ]
[ 1.          2.          0.95652174  0.        ]
[-2.7080502   0.0061034   0.95652174  0.        ]
-1.74542506037
0.14862516123
4 No Transaction
----
[-2.7080502  0.0030517  1.

0.12

In [14]:
g(-3.708 + 1)

0.062502941535436882

In [15]:
1.0/16.0

0.0625

In [16]:
g_inv(0.01)/float(1*24*30)

-0.0063821109029647081